This notebook presents the final calculation from the 
[growth_with_boundary_layer](./growth_with_boundary_layer.ipynb) notebook
but with the problem formally cast in terms of a pair of coupled ODEs solved
as an initial value problem using `scipy.integrate.solve_ivp`. The equations 
are briefly restated first.

We seek to find how the location of the particle ($l_p$) and it's radius ($r_p$) change with time.
The change in location is given by the Stokes equation: 

$$ \frac{d l_p}{d t } = \frac{2}{9} \frac{\Delta \rho g r_p^2}{\mu}$$

and the growth rate is determind from the liquid composition at the interface ($x_p$) and how fast oxygen
can be diffuse away from the interface:

$$\frac{\mathrm{d} r_p}{\mathrm{d} t} = \frac{D_l}{x_p} \left. \frac{\partial x}{\partial r}\right|_{r=r_p}  $$

The composition, $x$, is given by:

$$ x(r) = \begin{cases} 
      1.0 & r \leq r_p  \\
      (x_p - x_l)(r - r_p)/\delta + x_p & r_p\leq r\leq r_p + \delta \\
      x_l & r_p + \delta \leq r 
   \end{cases}$$
   
and we constrain the total composition around the particle (including the particle itself)
to match some imposed initial composition $x_i$:

$$ \frac{4}{3}\pi r^3 x_i = \int_0^R 4 \pi r^2 x(r) \mathrm{d}r$$

The thickness of the boundary layer depends on the falling velocity and radius of the particle in
a way that depends on $Re$:

$$ \delta = \begin{cases}
   2 r_p & Re \leq 10^{-2} \\
   2 r_p Pe^{-1/3} & 10^{-2} \leq Re \leq 10^2 \\
   2 r_p Re^{-1/2} Sc^{-1/3} & 200 \leq Re
   \end{cases} $$

with $Pe = (2r \frac{d l_p}{d t }) / D_l$, $Sc = \mu / D_l$ and $Re = (2 r_p \frac{d l_p}{d t }) / \mu$.

But then we have the whole thermodynamics stuff to put in two, which makes it hard to write down. Let's just 
try the code...



In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import numba
import scipy.integrate as spi
import scipy.optimize as spo

import growth
import falling
import feo_thermodynamics

Pr =  0.09075  Sc =  999.9999999999999


In [23]:
@numba.vectorize()
def x_r_well_mixed (r, rp, xl):
    if r < rp:
        x = 1.0
    else:
        x = xl
    return x

@numba.jit()
def intergrand_well_mixed(r, rp, xl):
    return 4.0 * np.pi * r**2 * x_r_well_mixed(r, rp, xl)

def total_composition_well_mixed(rp, xl, r_max):
    y, err, infodict = spi.quad(intergrand_well_mixed, 0.0, r_max, args=(rp, xl), full_output=1)
    
    return y

def _total_composition_well_mixed_error(xl, rp, xi, rtot):
    """
    For a given xl compute the difference between the composition and the pure
    liquid composition. When this is zero we have a consistent solution
    """
    # Calculate total composition for this configuration
    xtot = total_composition_well_mixed(rp, xl, rtot)
    xtot_pure_melt = 4/3 * np.pi * rtot**3 * xi
    error = xtot - xtot_pure_melt
    return error

@numba.vectorize()
def x_r (r, rp, delta, xl, xp):
    if r < rp:
        comp = 1.0
    elif r < (rp + delta):
        comp = (xl - xp)*(r - rp)/delta + xp
    else:
        comp = xl
    return comp

@numba.jit()
def intergrand(r, rp, delta, xl, xp):
    return 4.0 * np.pi * r**2 * x_r(r, rp, delta, xl, xp)

def total_composition(rp, delta, xl, xp, r_max):
    # specifing function discontiunities as breakpoints reduces errors here.
    print('in total composition with', rp, delta, xl, xp, r_max)
    y, err = spi.quad(intergrand, 0.0, r_max, args=(rp, delta, xl, xp), points=(rp, rp+delta))
    print('done returning ', y, err)
    return y, err

def _total_composition_error(xp, rp, xi, delta, rtot, temperature, pressure, dl, k0, debug=False):
    """
    For a given cl compute the difference between the composition and the pure
    liquid composition. When this is zero we have a consistent solution
    """
    # Compute growth rate for this composition 
    v = growth.growth_velocity_feo(xp, pressure, temperature, k0)
    # This gives us the composition at the edge of the boundary layer
    # because the graident at the boundary (and hence in the layer)
    # is set by the expulsion rate of O from the growing Fe
    print(xp, delta, dl, v)
    xl = xp + (delta*xp)/dl * v # Check sign here - it's negative in the notes.
    # but oxygen content has changed sign.
    # Calculate total composition for this configuration
    xtot, integration_error = total_composition(rp, delta, xl, xp, rtot)
    xtot_pure_melt = 4/3 * np.pi * rtot**3 * xi
    error = xtot - xtot_pure_melt
    if debug:
        print("Composition error:", error, "intgration error:", integration_error)
    return error

def diffusion_growth_rate(rp, xi, delta, rtot, temperature, pressure, dl, k0, debug=False):
    """
    Compute growth rate of a particle of Fe from an FeO liquid accounting for a diffusional boundary layer
    
    For an Fe particle of radius rp (m) in a spherical container of radius rtot (m)
    calculate drp/dt (in m/s) assuming the presence of a linear boundary layer of
    thickness delta (m) and total composition ci (mol frac Fe) pressute (in GPa) and
    temperature (K). We also need two material properties, the diffusivity of FeO in 
    the liquid (dl, in m^2s^-1) and prefactor for growth (k0, in m/s). We also need
    an initial guess for the liquid composition next to the particle (cl_guess).
    
    Returns the growth rate, the self-consistent composition at the interface
    and the self consistent composition at the outer side of the boundary layer.
    """
    xp, root_result = spo.brentq(_total_composition_error, 1.0E-12, 1.0-1.0E-12, 
                                 args=(rp, xi, delta, rtot, temperature, pressure, dl, k0, debug),
                                 xtol=2.0e-14, disp=True, full_output=True)
    if debug:
        print(root_result)
    v = growth.growth_velocity_feo(xp, pressure, temperature, k0)
    xl = xp + (delta*xp)/dl * v
    error = _total_composition_error(xp, rp, xi, delta, rtot, temperature, pressure, dl, k0)
    return v, xp, xl, error

In [ ]:
def dy_by_dt(t, y, xi, rtot, temperature, pressure, dl, k0, g, mu):
    """
    Find the growth rate and sinking rate of a particle at time t
    
    This is to be used with scipy.integrate.solve_ivp. t is the time
    (not used, everything is constant in time), y[0] is the current 
    particle radius and y[1] is the current height above the ICB.
    Returns growth rate of particle and vertical velocity of particle
    (positive is upwards). 
    """
    rp = y[0]
    z = y[1]
    print(t, 's, at z=', z, ',rp = ', rp)
    # Find liquid composition assuming no boundary layer (or we could do the whole thing
    # inside yet another self conssitent loop - no thanks).
    xl_no_bl = spo.brentq(_total_composition_well_mixed_error, 0.00000001, 0.999999999, 
                          args=(rp, xi, rtot))
    
    # Density calculation (with composition at previous step...)
    rho_liq, _, _, rho_hcp, _, _ = feo_thermodynamics.densities(xl_no_bl, pressure, temperature)
    delta_rho = rho_hcp - rho_liq
    
    # Falling velocity
    v_falling  = spo.brentq(falling.fzhang_opt, -1.0, 100.0, 
                             args=(rp, mu, g, delta_rho, rho_liq))
    print(v_falling)
    # Boundary layers
    _, _, _, _, delta, _, _ = falling.calculate_boundary_layers([rp], mu, g, delta_rho, rho_liq, 
                                                    1.0, dl, radius_top_flayer, radius_inner_core)
    
    delta = delta[0] # Proper vectoriziation of calc boundary layers needed
    print(delta)
    # Particle growth rate
    v, xp, xl, error = diffusion_growth_rate(rp, xi, delta, rtot, temperature, pressure, dl, k0)
    print(v, xp, xl, error)
    return [v, v_falling]

In [ ]:
r0 = 1.0E-10 # initial radius, m
t = 5000.0 # temperature, K
p = 330.0 # GPa
xi = 0.95 # Overall composition, mol frac Fe
k0 = 150.0 # growth rate prefactor, m/s
rtot = 1.0 # initial box radius
dl = 1.0E-9 # diffusion
radius_inner_core = 1221.0e3
radius_top_flayer = radius_inner_core + 200.0e3
mu = 1.0e-6 # kinematic viscosity
g = 3.7 # ICB gravity - from PREM

sol = spi.solve_ivp(dy_by_dt, [0, 10], [r0, radius_top_flayer], args=(xi, rtot, t, p, dl, k0, g, mu))

0.0 s, at z= 1421000.0 ,rp =  1e-10
2.478861462484882e-14
2e-10
+++ 1e-12 330.0 5000.0 150.0
-117226277604281.14
1e-12 2e-10 1e-09 -117226277604281.14
>>> -23.44525552085523
in total composition with 1e-10 2e-10 -23.44525552085523 1e-12 1.0
done returning  -98.20725667447243 1.0903195754527187e-12
-98.20725667447243
Composition error: -102.1866073690195 intgration error: 1.0903195754527187e-12
+++ 0.999999999999 330.0 5000.0 150.0
32.774131170046154
0.999999999999 2e-10 1e-09 32.774131170046154
>>> 7.554826234001676
in total composition with 1e-10 2e-10 7.554826234001676 0.999999999999 1.0
done returning  31.645582127849476 3.513365390600484e-13
31.645582127849476
Composition error: 27.666231433302407 intgration error: 3.513365390600484e-13
+++ 0.7869416511140439 330.0 5000.0 150.0
1.0361318294076327
0.7869416511140439 2e-10 1e-09 1.0361318294076327
>>> 0.9500167096352155
in total composition with 1e-10 2e-10 0.9500167096352155 0.7869416511140439 1.0
done returning  3.979420687703387 4

Composition error: 66.14283925589405 intgration error: 7.785126982011848e-13
+++ 0.7815058226082247 330.0 5000.0 150.0
1.1697771418184288e-06
0.7815058226082247 0.00018432254656031914 1e-09 1.1697771418184288e-06
>>> 0.9500112178266673
in total composition with 0.002952595641615335 0.00018432254656031914 0.9500112178266673 0.7815058226082247 1.0
done returning  3.979397687286134 4.4180189365651644e-14
3.979397687286134
Composition error: 4.699273906316748e-05 intgration error: 4.4180189365651644e-14
+++ 0.7815058226078189 330.0 5000.0 150.0
1.1696998536425696e-06
0.7815058226078189 0.00018432254656031914 1e-09 1.1696998536425696e-06
>>> 0.9500000845306484
in total composition with 0.002952595641615335 0.00018432254656031914 0.9500000845306484 0.7815058226078189 1.0
done returning  3.97935105224614 4.4179671612700096e-14
3.97935105224614
Composition error: 3.57699069475359e-07 intgration error: 4.4179671612700096e-14
+++ 0.7815058226078085 330.0 5000.0 150.0
1.1696973223340734e-06
0.781

+++ 1e-12 330.0 5000.0 150.0
-117226277604281.14
1e-12 0.00010527420962115176 1e-09 -117226277604281.14
>>> -12340903.72162042
in total composition with 9.114797080960489e-05 0.00010527420962115176 -12340903.72162042 1e-12 1.0
done returning  -51693456.62716128 5.739126576993633e-07
-51693456.62716128
Composition error: -51693460.60651197 intgration error: 5.739126576993633e-07
+++ 0.999999999999 330.0 5000.0 150.0
32.774131170046154
0.999999999999 0.00010527420962115176 1e-09 32.774131170046154
>>> 3450271.7549431124
in total composition with 9.114797080960489e-05 0.00010527420962115176 3450271.7549431124 0.999999999999 1.0
done returning  14452464.530908138 1.6045458884792628e-07
14452464.530908138
Composition error: 14452460.551557442 intgration error: 1.6045458884792628e-07
+++ 0.7815063982999416 330.0 5000.0 150.0
0.00011166633606118914
0.7815063982999416 0.00010527420962115176 1e-09 0.00011166633606118914
>>> 9.968571502668336
in total composition with 9.114797080960489e-05 0.000

done returning  3.9793507938544614 4.417966874397618e-14
3.9793507938544614
Composition error: 9.930739075514339e-08 intgration error: 4.417966874397618e-14
+++ 0.781505827183804 330.0 5000.0 150.0
2.048000902199476e-06
0.781505827183804 0.00010527422416162165 1e-09 2.048000902199476e-06
>>> 0.9499998168215195
in total composition with 9.114908732727779e-05 0.00010527422416162165 0.9499998168215195 0.781505827183804 1.0
done returning  3.9793499272491593 4.4179659122724586e-14
3.9793499272491593
Composition error: -7.672979114303757e-07 intgration error: 4.4179659122724586e-14
      converged: True
           flag: 'converged'
 function_calls: 6
     iterations: 5
           root: 0.7815058271838143
+++ 0.7815058271838143 330.0 5000.0 150.0
2.048003416854627e-06
0.7815058271838143 0.00010527422416162165 1e-09 2.048003416854627e-06
>>> 0.9500000237082958
in total composition with 9.114908732727779e-05 0.00010527422416162165 0.9500000237082958 0.7815058271838143 1.0
done returning  3.979

+++ 0.7815063982999397 330.0 5000.0 150.0
0.00011166633606118914
0.7815063982999397 0.00010527422900848287 1e-09 0.00011166633606118914
>>> 9.968573194561074
in total composition with 9.114945949973633e-05 0.00010527422900848287 9.968573194561074 0.7815063982999397 1.0
done returning  41.75626175294457 4.635876322039386e-13
41.75626175294457
Composition error: 37.776911058397495 intgration error: 4.635876322039386e-13
+++ 0.781505827184222 330.0 5000.0 150.0
2.0480815543511e-06
0.781505827184222 0.00010527422900848287 1e-09 2.0480815543511e-06
>>> 0.9500064600275168
in total composition with 9.114945949973633e-05 0.00010527422900848287 0.9500064600275168 0.781505827184222 1.0
done returning  3.979377754245369 4.417996806444356e-14
3.979377754245369
Composition error: 2.7059698298526058e-05 intgration error: 4.417996806444356e-14
+++ 0.7815058271838129 330.0 5000.0 150.0
2.0480025841873584e-06
0.7815058271838129 0.00010527422900848287 1e-09 2.0480025841873584e-06
>>> 0.9499999629602703


...

2.0479891449376453e-06
0.7815058271837404 0.00010527495625844281 1e-09 2.0479891449376453e-06
>>> 0.9500000212553368
in total composition with 9.120528526807917e-05 0.00010527495625844281 0.9500000212553368 0.7815058271837404 1.0
done returning  3.979350783579531 4.417966862990154e-14
3.979350783579531
Composition error: 8.903246051517044e-08 intgration error: 4.417966862990154e-14
+++ 0.7815058271837301 330.0 5000.0 150.0
2.047987462949763e-06
0.7815058271837301 0.00010527495625844281 1e-09 2.047987462949763e-06
>>> 0.9499998828731491
in total composition with 9.120528526807917e-05 0.00010527495625844281 0.9499998828731491 0.7815058271837301 1.0
done returning  3.979350203925579 4.41796621944499e-14
3.979350203925579
Composition error: -4.906214918776186e-07 intgration error: 4.41796621944499e-14
      converged: True
           flag: 'converged'
 function_calls: 6
     iterations: 5
           root: 0.7815058271837404
+++ 0.7815058271837404 330.0 5000.0 150.0
2.0479891449376453e-06
0

2.0480353413177e-06
0.7815058271839807 0.00010527662788254484 1e-09 2.0480353413177e-06
>>> 0.9500064974905784
in total composition with 9.133347570350628e-05 0.00010527662788254484 0.9500064974905784 0.7815058271839807 1.0
done returning  3.9793779111702707 4.4179969806659955e-14
3.9793779111702707
Composition error: 2.72166232000437e-05 intgration error: 4.4179969806659955e-14
+++ 0.7815058271835692 330.0 5000.0 150.0
2.0479563711539583e-06
0.7815058271835692 0.00010527662788254484 1e-09 2.0479563711539583e-06
>>> 0.9500000002752811
in total composition with 9.133347570350628e-05 0.00010527662788254484 0.9500000002752811 0.7815058271835692 1.0
done returning  3.979350695698475 4.417966765422582e-14
3.979350695698475
Composition error: 1.1514043052329725e-09 intgration error: 4.417966765422582e-14
+++ 0.7815058271835589 330.0 5000.0 150.0
2.0479538564988076e-06
0.7815058271835589 0.00010527662788254484 1e-09 2.0479538564988076e-06
>>> 0.9499997933837809
in total composition with 9.133

32.774131170046154
0.999999999999 0.00010529648270914652 1e-09 32.774131170046154
>>> 3451001.7360506137
in total composition with 9.284266481084761e-05 0.00010529648270914652 3451001.7360506137 0.999999999999 1.0
done returning  14455522.268619293 1.6048853655602817e-07
14455522.268619293
Composition error: 14455518.289268598 intgration error: 1.6048853655602817e-07
+++ 0.7815063982976845 330.0 5000.0 150.0
0.00011166590342392979
0.7815063982976845 0.00010529648270914652 1e-09 0.00011166590342392979
>>> 9.970479627839051
in total composition with 9.284266481084761e-05 0.00010529648270914652 9.970479627839051 0.7815063982976845 1.0
done returning  41.76424740198116 4.636762907182086e-13
41.76424740198116
Composition error: 37.784896707434086 intgration error: 4.636762907182086e-13
+++ 0.7815058271819658 330.0 5000.0 150.0
2.047648067771135e-06
0.7815058271819658 0.00010529648270914652 1e-09 2.047648067771135e-06
>>> 0.9500064074932602
in total composition with 9.284266481084761e-05 0.0

+++ 0.7815058271810553 330.0 5000.0 150.0
2.047473324218174e-06
0.7815058271810553 0.00010530142727870634 1e-09 2.047473324218174e-06
>>> 0.9499999397442778
in total composition with 9.321474865257218e-05 0.00010530142727870634 0.9499999397442778 0.7815058271810553 1.0
done returning  3.979350442146757 4.417966483923627e-14
3.979350442146757
2.047473324218174e-06 0.7815058271810553 0.9499999397442778 -2.5240031353845893e-07
1.0104452566915154 s, at z= 1421000.0002359492 ,rp =  9.32147512250463e-05
0.00023814033916796942
0.00010530142731295915
+++ 1e-12 330.0 5000.0 150.0
-117226277604281.14
1e-12 0.00010530142731295915 1e-09 -117226277604281.14
>>> -12344094.35031598
in total composition with 9.32147512250463e-05 0.00010530142731295915 -12344094.35031598 1e-12 1.0
done returning  -51706821.501381226 5.740610376101653e-07
-51706821.501381226
Composition error: -51706825.48073192 intgration error: 5.740610376101653e-07
+++ 0.999999999999 330.0 5000.0 150.0
32.774131170046154
0.9999999999

2.0422706192846363e-06
0.7815058271539483 0.00010556979266417878 1e-09 2.0422706192846363e-06
>>> 0.9500001135860351
in total composition with 0.00011159792490259872 0.00010556979266417878 0.9500001135860351 0.7815058271539483 1.0
done returning  3.979351170332957 4.417967292372712e-14
3.979351170332957
Composition error: 4.757858862447506e-07 intgration error: 4.417967292372712e-14
      converged: True
           flag: 'converged'
 function_calls: 6
     iterations: 5
           root: 0.781505827153938
+++ 0.781505827153938 330.0 5000.0 150.0
2.0422689539500993e-06
0.781505827153938 0.00010556979266417878 1e-09 2.0422689539500993e-06
>>> 0.9499999761902476
in total composition with 0.00011159792490259872 0.00010556979266417878 0.9499999761902476 0.781505827153938 1.0
done returning  3.9793505948108274 4.417966653414793e-14
3.9793505948108274
2.0422689539500993e-06 0.781505827153938 0.9499999761902476 -9.97362432642035e-08
10.0 s, at z= 1421000.0028166946 ,rp =  0.00011159788847061977